<a href="https://colab.research.google.com/github/DimasVeliz/MachineLearning6hCourse/blob/main/ClasificationDNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
%tensorflow_version 2.x

In [19]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd


In [20]:
CSV_COLUMNS_NAMES= ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [21]:
train_path =tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

In [22]:
test_path =tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

In [23]:
train =pd.read_csv(train_path, names=CSV_COLUMNS_NAMES,header=0)
test =pd.read_csv(test_path, names=CSV_COLUMNS_NAMES,header=0)

print(len(test))
print(test)

30
    SepalLength  SepalWidth  PetalLength  PetalWidth  Species
0           5.9         3.0          4.2         1.5        1
1           6.9         3.1          5.4         2.1        2
2           5.1         3.3          1.7         0.5        0
3           6.0         3.4          4.5         1.6        1
4           5.5         2.5          4.0         1.3        1
5           6.2         2.9          4.3         1.3        1
6           5.5         4.2          1.4         0.2        0
7           6.3         2.8          5.1         1.5        2
8           5.6         3.0          4.1         1.3        1
9           6.7         2.5          5.8         1.8        2
10          7.1         3.0          5.9         2.1        2
11          4.3         3.0          1.1         0.1        0
12          5.6         2.8          4.9         2.0        2
13          5.5         2.3          4.0         1.3        1
14          6.0         2.2          4.0         1.0        1
15   

In [24]:
train_y=train.pop('Species')
test_y=test.pop('Species')


In [25]:
train.shape

(120, 4)

In [26]:
def input_fn(features, labels, training=True, batch_size=256):
  dataset =tf.data.Dataset.from_tensor_slices((dict(features),labels))
  if training:
    dataset= dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [27]:
my_feature_columns=[]
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [28]:
classifier= tf.estimator.DNNClassifier(feature_columns=my_feature_columns, hidden_units=[30,10], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpvoivuzu_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [38]:
classifier.train(
                input_fn=lambda:input_fn(train,train_y,training=True),
                 steps=5000
                 )

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvoivuzu_/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into

In [39]:
eval_result=classifier.evaluate(input_fn=lambda:input_fn(test,test_y,training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-11-23T02:36:40Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvoivuzu_/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.18128s
INFO:tensorflow:Finished evaluation at 2020-11-23-02:36:40
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.73333335, aver

In [44]:
def input_fn(features,batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features =['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict ={}
print("Pleast type a numeric value as prompted:")

for feature in features:
  valid=True
  while valid:
    val=input(feature + ": ")
    if not val.isdigit():valid= False
  predict[feature]=[float(val)]

predictions= classifier.predict(input_fn=lambda:input_fn(predict))

for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability= pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id],100*probability))


Pleast type a numeric value as prompted:
SepalLength: 2.4
SepalWidth: 2.6
PetalLength: 6.5
PetalWidth: 6.3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpvoivuzu_/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virginica" (96.5%)
